In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/test.csv')

In [ ]:
!mkdir images_train
!mkdir images_train/0
!mkdir images_train/1
!mkdir images_train/2
!mkdir images_train/3
!mkdir images_train/4
!mkdir images_train/5
!mkdir images_train/6
!mkdir images_train/7
!mkdir images_train/8
!mkdir images_train/9
!mkdir images_test

In [ ]:
import cv2

for idx in range(len(train)) :
    img = train.loc[idx, '0':].values.reshape(28, 28).astype(int)
    digit = train.loc[idx, 'digit']
    cv2.imwrite(f'./images_train/{digit}/{train["id"][idx]}.png', img)

In [ ]:
import cv2

for idx in range(len(test)) :
    img = test.loc[idx, '0':].values.reshape(28, 28).astype(int)
    cv2.imwrite(f'./images_test/{test["id"][idx]}.png', img)

In [ ]:
import tensorflow as tf
EfficientNetB1_model =  tf.keras.applications.EfficientNetB1(weights=None, include_top=True, input_shape=(240, 240, 1), classes=10)

In [ ]:
from tensorflow.keras.optimizers import Adam
EfficientNetB1_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.002,epsilon=None), metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2,
                             rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1)

train_generator = datagen.flow_from_directory('./images_train', target_size=(240,240), color_mode='grayscale', class_mode='categorical', subset='training')
val_generator = datagen.flow_from_directory('./images_train', target_size=(240,240), color_mode='grayscale', class_mode='categorical', subset='validation')

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/DACON_CVLC/Checkpoint/CVLC_06_EfficientNetB1(InputShape-240x240).h5', monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:
EfficientNetB1_model.summary()

In [ ]:
EfficientNetB1_model.fit_generator(train_generator, epochs=500, validation_data=val_generator, callbacks=[checkpoint])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(EfficientNetB1_model.history.history["accuracy"], label='EfficientNetB1_acc')
plt.plot(EfficientNetB1_model.history.history["val_accuracy"], label='EfficientNetB1_val')

plt.legend()
plt.show()

In [ ]:
EfficientNetB1_model = tf.keras.models.load_model('/content/drive/MyDrive/DACON_CVLC/Checkpoint/CVLC_06_EfficientNetB1(InputShape-240x240).h5', compile=False)

In [ ]:
!mkdir images_test/none
!mv images_test/*.png images_test/none

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_directory('./images_test', target_size=(240,240), color_mode='grayscale', class_mode='categorical', shuffle=False)

In [ ]:
EfficientNetB1_predict = EfficientNetB1_model.predict_generator(test_generator).argmax(axis=1)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_CVLC/data/submission.csv')
submission.head()

In [ ]:
import numpy as np
mylist = []

for i in range(len(submission)):
    name =  test_generator.filenames
    id = name[i].split('/')[1].rstrip('.').split('.')[0]
    mylist.append(id)
print(mylist)

In [ ]:
for i in range(len(submission)):
    submission["id"][i] = mylist[i]

In [ ]:
submission["EfficientNetB1_predict"] = EfficientNetB1_predict
submission.head()

In [ ]:
from collections import Counter

for i in range(len(submission)) :
    predicts = submission.loc[i, ['EfficientNetB1_predict']]
    submission.at[i, "digit"] = Counter(predicts).most_common(n=1)[0][0]

submission.head()

In [ ]:
submission = submission[['id', 'digit']]
submission.head()

In [ ]:
from google.colab import files

submission.to_csv('/content/drive/MyDrive/DACON_CVLC/Submission/EfficientNetB1_model(InputShape-240x240).csv', index=False)
files.download('/content/drive/MyDrive/DACON_CVLC/Submission/EfficientNetB1_model(InputShape-240x240).csv')